# Machine translation Pytorch

In [1]:
import numpy as np
import pandas as pd
import csv
import re
import imblearn
import torch
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.parsing.porter import PorterStemmer
from gensim.utils import tokenize
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch
from torchtext.datasets import IWSLT2017
from torchtext.legacy import data
from torchtext.vocab import Vocab
# from torchtext import data
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import roc_curve,auc
from numpy import interp
from itertools import cycle
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from collections import Counter
from functools import reduce
# ! pip install captum bokeh spacy emot parameter-sherpa
import sherpa
import captum
import spacy
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization, IntegratedGradients, LayerConductance
from captum.attr import visualization as viz
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
import emot
from bokeh.io import output_notebook
output_notebook()
# ! python -m spacy download en_core_web_sm
# !python -m spacy download de
%matplotlib inline

Loading BokehJS ...

## Text Preprocessing

In [2]:
spacy_german = spacy.load("de")
spacy_english = spacy.load("en")

def tokenize_german(text):
    return [token.text for token in spacy_german.tokenizer(text)]

def tokenize_english(text):
    return [token.text for token in spacy_english.tokenizer(text)]

train_iter, valid_iter, test_iter, = IWSLT2017(root='.pytorch/.data/', language_pair=('en','de'))

en_counter = Counter()
de_counter = Counter()

for (de, en) in train_iter:
    en_counter.update(tokenize_english(en))
    de_counter.update(tokenize_german(de))

german = Vocab(de_counter, min_freq=10, specials=('<unk>', '<BOS>', '<EOS>', '<PAD>'))
english = Vocab(en_counter, min_freq=10, specials=('<unk>', '<BOS>', '<EOS>', '<PAD>'))


INFO:root:File C:\Users\User\YandexDisk\myAI\myNLP\deepLearningRecipes\.pytorch\.data\2017-01-trnmted.tgz already exists.
INFO:root:Validating hash aca701032b1c4411afc4d9fa367796ba matches hash of C:\Users\User\YandexDisk\myAI\myNLP\deepLearningRecipes\.pytorch\.data\2017-01-trnmted.tgz
INFO:root:Opening tar file C:\Users\User\YandexDisk\myAI\myNLP\deepLearningRecipes\.pytorch\.data\2017-01-trnmted.tgz.
INFO:root:C:\Users\User\YandexDisk\myAI\myNLP\deepLearningRecipes\.pytorch\.data\2017-01-trnmted/._texts.html already extracted.
INFO:root:C:\Users\User\YandexDisk\myAI\myNLP\deepLearningRecipes\.pytorch\.data\2017-01-trnmted/texts.html already extracted.
INFO:root:C:\Users\User\YandexDisk\myAI\myNLP\deepLearningRecipes\.pytorch\.data\2017-01-trnmted/texts/DeEnItNlRo/DeEnItNlRo/._.eval already extracted.
INFO:root:C:\Users\User\YandexDisk\myAI\myNLP\deepLearningRecipes\.pytorch\.data\2017-01-trnmted/texts/DeEnItNlRo/DeEnItNlRo/.eval already extracted.
INFO:root:C:\Users\User\YandexDisk\

In [3]:
print(f'Unique tokens in source (de) vocabs {len(german)}')
print(f'Unique tokens in source (en) vocabs {len(english)}')

Unique tokens in source (de) vocabs 14101
Unique tokens in source (en) vocabs 17069


In [4]:
english_transform = lambda x: [english['<BOS>']] + [english[token] for token in tokenize_english(x)] + [english['<EOS>']]
german_transform = lambda x: [german['<BOS>']] + [german[token] for token in tokenize_german(x)] + [german['<EOS>']]
print("output of the text_transform:", english_transform("here is an example"))

output of the text_transform: [1, 0, 7366, 39, 0, 2]
